In [3]:
import numpy as np
import cv2
import time
import os
import random

# Запуск

In [31]:
#На входе нам нужны:
#1. Структура видео:
#- Путь к видео файлу 1, имя класса 1, нужен ли поворот (если 0 то не нужен)
#- Путь к видео файлу 2, имя класса 2, нужен ли поворот (если 0 то не нужен)
#- Путь к видео файлу N, имя класса N, нужен ли поворот (если 0 то не нужен)
_VIDEO =  [
    ["/Users/alexeyvaganov/doc/files/foto_mp/video/mam.mov","mam",90],
    ["/Users/alexeyvaganov/doc/files/foto_mp/video/pap.mov","pap",90]
]

#2.Путь к папке в которой будут созданы файлы
OUT_FOLDER = "/Users/alexeyvaganov/doc/files/foto_mp/files"

#3.Размер выходных файлов
File_size = [320,240]
#4.Надо ли проводить аугментацию
AUG = False
#5 Процент файлов Validation
_VALID = 20
#6 Процент файлов Test
_TEST = 10

start_video_to_frame()

Папка /Users/alexeyvaganov/doc/files/foto_mp/files/train существует
Папка /Users/alexeyvaganov/doc/files/foto_mp/files/validation существует
Папка /Users/alexeyvaganov/doc/files/foto_mp/files/test существует
Папка /Users/alexeyvaganov/doc/files/foto_mp/files/train/mam существует
Папка /Users/alexeyvaganov/doc/files/foto_mp/files/validation/mam существует
Папка /Users/alexeyvaganov/doc/files/foto_mp/files/train/pap существует
Папка /Users/alexeyvaganov/doc/files/foto_mp/files/validation/pap существует


In [30]:
def start_video_to_frame():
    
    # создадим папки
    mk_dir_(os.path.join(OUT_FOLDER, "train"))
    mk_dir_(os.path.join(OUT_FOLDER, "validation"))
    mk_dir_(os.path.join(OUT_FOLDER, "test"))
    
    # создадим папки меток
    for i in _VIDEO:
       mk_dir_(os.path.join(OUT_FOLDER, "train/%s" %i[1] ))
       mk_dir_(os.path.join(OUT_FOLDER, "validation/%s" %i[1] ))
    
    # сначала все нарезаем в папку train
    # а после тогот как все нарежем часть случайных файлов перенесем в остальные папки
    
      

# Функции

In [ ]:
# функции нарезки фото из видео
# функция нарезает каждый фрэйм
# разворачивает и уменьшает картинку

def video_to_frames(video, prefix):
    # extract frames from a video and save to directory as 'x.jpg' where 
    # x is the frame index
    #video = cv2.VideoCapture('/Users/alexeyvaganov/doc/files/foto_mp/pap.mov')
    vidcap = cv2.VideoCapture(video)
    count = 0
    while vidcap.isOpened():
        success, image = vidcap.read()
        if success:
           # img_name = os.path.join(path_output_dir, 'img_%s_%d_0.jpg') % (prefix,count)
            img_name = 'img_%s_%d.jpg' % (prefix,count)
            img_path = return_file_name(img_name)
            cv2.imwrite(img_path,image) 
            convert_image(img_path)
            #augmentation(img_name)
            count += 1
        else:
            break
    # print(count)
    cv2.destroyAllWindows()
    vidcap.release()

def convert_image(image_path): # конвертировать картинку
    
    image = cv2.imread(image_path)
    
    #scale_percent = 12# Процент от изначального размера
    #width = int(image.shape[1] * scale_percent / 100)
    #height = int(image.shape[0] * scale_percent / 100)
    
    # УКАЗАТЬ РАЗМЕР
    width = 230
    height = 129
    # УКАЗАТЬ РАЗМЕР
    
    dim = (width, height)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    
    # и тут же перевренем
    image = rotate_bound(image, 90)

    cv2.imwrite(image_path, image)
    
def rotate_bound(image, angle): # Повернуть картинку. параметры: картинка, угол
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    # perform the actual rotation and return the image

    return cv2.warpAffine(image, M, (nW, nH))    

In [ ]:
# функции аугментацию
def augmentation(img_name):
    
    img_name_ws = img_name.replace(".jpg","_")
    
    img = cv2.imread(return_file_name(img_name))
    
    img_aug = rotation(img,30)
    cv2.imwrite(return_file_name(img_name_ws + "r+.jpg"),img_aug)
    
    img_aug = rotation(img,-30)
    cv2.imwrite(return_file_name(img_name_ws + "r-.jpg"),img_aug)
    
    img_aug = vertical_flip(img, True)
    cv2.imwrite(return_file_name(img_name_ws + "v.jpg"),img_aug)
    
    img_aug = horizontal_flip(img, True)
    cv2.imwrite(return_file_name(img_name_ws + "t.jpg"),img_aug)
    
    img_aug = channel_shift(img, 50)
    cv2.imwrite(return_file_name(img_name_ws + "c.jpg"),img_aug)
    
    img_aug = zoom(img, 0.2)
    cv2.imwrite(return_file_name(img_name_ws + "z2.jpg"),img_aug)
    
    img_aug = zoom(img, 0.3)
    cv2.imwrite(return_file_name(img_name_ws + "z3.jpg"),img_aug)
    
    img_aug = brightness(img, 0.5, 3)
    cv2.imwrite(return_file_name(img_name_ws + "b.jpg"),img_aug)
    
    img_aug = vertical_shift(img, 0.7)
    cv2.imwrite(return_file_name(img_name_ws + "vs.jpg"),img_aug)
    
    img_aug = horizontal_shift(img, ratio=0.7)
    cv2.imwrite(return_file_name(img_name_ws + "gs.jpg"),img_aug)     

In [ ]:
# вспомогательные функции для аугментации
# https://towardsdatascience.com/complete-image-augmentation-in-opencv-31a6b02694f5

# горизонтальное смещение
def horizontal_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = w*ratio
    if ratio > 0:
        img = img[:, :int(w-to_shift), :]
    if ratio < 0:
        img = img[:, int(-1*to_shift):, :]
    img = fill(img, w, h)
    return img

# вертикальное смещение
def vertical_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :, :]
    if ratio < 0:
        img = img[int(-1*to_shift):, :, :]
    img = fill(img, w, h)
    return img

# яркость
def brightness(img, low, high):
    value = random.uniform(low, high)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype = np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value
    hsv[:,:,1][hsv[:,:,1]>255]  = 255
    hsv[:,:,2] = hsv[:,:,2]*value 
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype = np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img

# вспомогательная
def fill(img, h, w):
    img = cv2.resize(img, (h, w), cv2.INTER_CUBIC)
    return img

# зумиррование
def zoom(img, value):
    if value > 1 or value < 0:
        print('Value for zoom should be less than 1 and greater than 0')
        return img
    value = random.uniform(value, 1)
    h, w = img.shape[:2]
    h_taken = int(value*h)
    w_taken = int(value*w)
    h_start = random.randint(0, h-h_taken)
    w_start = random.randint(0, w-w_taken)
    img = img[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
    img = fill(img, w, h)
    return img

# смещение канала
def channel_shift(img, value):
    value = int(random.uniform(-value, value))
    img = img + value
    img[:,:,:][img[:,:,:]>255]  = 255
    img[:,:,:][img[:,:,:]<0]  = 0
    img = img.astype(np.uint8)
    return img

# горизонтальное отображение
def horizontal_flip(img, flag):
    if flag:
        return cv2.flip(img, 1)
    else:
        return img

# вертикальное отображение
def vertical_flip(img, flag):
    if flag:
        return cv2.flip(img, 0)
    else:
        return img

#  сдвиг
def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))
    return img

In [29]:
# вспомогательные
def return_file_name(img_name):
    return os.path.join(W_DIR, img_name)

def mk_dir_ (path):
    if os.path.isdir(path):
        print ("Папка %s существует" %path )
    else:
        os.mkdir(path)
        print ("Создана папка %s " %path )


In [ ]:
W_DIR = "/Users/alexeyvaganov/doc/files/foto_mp/mam/"
File_Name = "mam.mov"
prefix = "mam"

video_to_frames(os.path.join(W_DIR, File_Name),prefix)

In [ ]:
W_DIR = "/Users/alexeyvaganov/doc/files/foto_mp/pap/"
File_Name = "pap.mov"
prefix = "pap"

video_to_frames(os.path.join(W_DIR, File_Name),prefix)